# Portfolio Currator - Project 1 
welcome to my first project! this project serves to collect data, analyze the data, and create visualizations to support our conclusions and analysis. After getting the data, we will move on to using the data to offer a financial product, in the form of a portfolio optimized to meet medical insurance charges by a certain date. 

## Part 1. Importing Libraries and Dependencies

In [46]:
# Import the required libraries and dependencies
import os
import requests
import json
import pandas as pd
from pathlib import Path
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
from io import BytesIO
import numpy as np
import numba as nb
import holoviews as hv
import panel as pn
from scipy.optimize import minimize
%matplotlib inline

In [8]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv()

True

## Part 2. Getting the MD Insurance Data
Part one of the project is largely made up of pulling in the data, cleaing the data, and visualizing it for use later in the portfolio currator.
The primary goal of the Data creation segment of the project will be to have a dataframe with the expected percent change in medical costs, in relation to age.


In [9]:
# Import the data by reading in the CSV file 
# Review the first 5 rows of the DataFrame
medical_costs_df = pd.read_csv(
    Path("./Resources/Medical_Cost.csv"),
)
medical_costs_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
# Creating a Dataframe holding grouped based on charges, and age.

md_cost_age_df = (
 medical_costs_df[["charges", "age"]]
 .groupby("age")
 .mean()
 .sort_values("age")
)
# Reviewing dataframe
print(" 1. Review of MD Data :")
display(md_cost_age_df.head())


In [ ]:
# Creating summary statistics of the data.
print("2. Summary Statistics of MD Data :")
print("")

# Review the summary of the Medical Insurance data, before applying .pct_change(). 
print(" MD Insurance Charges - Data Summary :")
display(md_cost_age_df.describe())

md_cost_age_df_change = md_cost_age_df['charges'].pct_change()

# Review the MD cost 'pct_change' data frame.
print(" Rate of Change of MD Insurance Charges - Data Summary :")
print("")
display(md_cost_age_df_change.describe())

---

## Part 2.5 - Get Stock Data

We will be using the Alpaca API to pull in stock data, which we can later analyze to find the 'top ten' stocks in the list of '50 S&P500 top-performing stocks'. 

 - Why I decided to take the S&P 500's top 50 most weighted stocks as our data sample:
 

   1. The Alpaca API is a free-to-use online public API, which holds a large database of stock, and crypto data. We will, as mentioned, pull in 50 of the most highly weighted stocks in the S&P 500, which is generally regarded to have companies of such a size that any major downturns in the portfolio will likely correlate with downturns in the overall economy. More specifically, the S&P 500 is very much linked to U.S. economic prosperity. 
   
   2. If the U.S. economy continues to grow and stay the dominant player in our global economy, U.S. corporations andtheir stocks should generally produce stable returns. The U.S. has a strong currency used throughout the world, and a strong regulatory and business framework. 
    
   3. These are among the most important reasons for my picking the S&P 500's 50 most weighted stocks to begin our analysis-- this was not a quantitative decision, but I believe my explanation has adequately described why a focus on U.S. stocks is  
    
   4. U.S. economic growth has outpaced Europe, and the Chinese and other emerging markets. Though investment in emerging markets can prove lucrative, regulatory issues in many emerging markets (bureacracy in the Indian Business Environment and Government control in the Chinese economy, as well as inneficient monteary policies in many South American and African markets). Europe has ben outpaces by the U.S. for many years, and have seen themselves become less productive than Americans on-average. The U.S. has a well-developed regulatory framework with the SEC, IRA, and other related instutions overseeing the American financial markets. 


#### Step 1. Load .env files :

In [32]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv()

True

In [42]:
# Load .env environment variables
load_dotenv()

# Set Alpaca API key and secret key
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
type(alpaca_secret_key )

# Create the Alpaca REST object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version = "v2"
)

---

#### Step 2. Pull in stock data from Alpaca:

In [75]:
# Set the ticker information
tickers = [
    "BRK.B",
    "MSFT",
    "AMZN",
    "GOOGL",
    "MS",
    "NVDA",
    "TSLA",
    "GOOG",
    "XOM",
    "UNH",
    "JPM",
    "JNJ",
    "META",
    "V",
    "HD",
    "PG",
    "MA",
    "CVX",
    "MRK",
    "ABBV",
    "LLY",
    "BAC",
    "AVGO",
    "PEP",
    "PFE",
    "KO",
    "COST",
    "TMO",
    "CSCO",
    "WMT",
    "MCD",
    "DIS",
    "ABT",
    "WFC",
    "ACN",
    "CMCSA",
    "ADBE",
    "CRM",
    "DHR",
    "VZ",
    "TXN",
    "LIN",
    "NKE",
    "NFLX",
    "PM",
    "BMY",
    "NEE",
    "RTX",
    "QCOM",
    "T"
    ]
# Set timeframe to '1Day'
timeframe = "1Day"

# Set start and end datetimes between now and 3 years ago
start_date = pd.Timestamp("2017-05-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-05-01", tz="America/New_York").isoformat()

# Get price data for the S&P 500 top-50 stocks.
prices_df = alpaca.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df

# Reorganize the DataFrame
# Separate ticker data
BRK = prices_df[prices_df['symbol']=='BRK.B'].drop('symbol', axis=1)
MSFT = prices_df[prices_df['symbol']=='MSFT'].drop('symbol', axis=1)
AMZN = prices_df[prices_df['symbol']=='AMZN'].drop('symbol', axis=1)
GOOGL = prices_df[prices_df['symbol']=='GOOGL'].drop('symbol', axis=1)
MS = prices_df[prices_df['symbol']=='MS'].drop('symbol', axis=1)
NVDA = prices_df[prices_df['symbol']=='NVDA'].drop('symbol', axis=1)
TSLA = prices_df[prices_df['symbol']=='TSLA'].drop('symbol', axis=1)
GOOG = prices_df[prices_df['symbol']=='GOOG'].drop('symbol', axis=1)
XOM = prices_df[prices_df['symbol']=='XOM'].drop('symbol', axis=1)
UNH = prices_df[prices_df['symbol']=='UNH'].drop('symbol', axis=1)
JPM = prices_df[prices_df['symbol']=='JPM'].drop('symbol', axis=1)
JNJ = prices_df[prices_df['symbol']=='JNJ'].drop('symbol', axis=1)
META = prices_df[prices_df['symbol']=='META'].drop('symbol', axis=1)
V = prices_df[prices_df['symbol']=='V'].drop('symbol', axis=1)
HD = prices_df[prices_df['symbol']=='HD'].drop('symbol', axis=1)
PG = prices_df[prices_df['symbol']=='PG'].drop('symbol', axis=1)
MA = prices_df[prices_df['symbol']=='MA'].drop('symbol', axis=1)
CVX = prices_df[prices_df['symbol']=='CVX'].drop('symbol', axis=1)
MRK = prices_df[prices_df['symbol']=='MRK'].drop('symbol', axis=1)
ABBV = prices_df[prices_df['symbol']=='ABBV'].drop('symbol', axis=1)
LLY = prices_df[prices_df['symbol']=='LLY'].drop('symbol', axis=1)
BAC = prices_df[prices_df['symbol']=='BAC'].drop('symbol', axis=1)
AVGO = prices_df[prices_df['symbol']=='AVGO'].drop('symbol', axis=1)
PEP = prices_df[prices_df['symbol']=='PEP'].drop('symbol', axis=1)
PFE = prices_df[prices_df['symbol']=='PFE'].drop('symbol', axis=1)
KO = prices_df[prices_df['symbol']=='KO'].drop('symbol', axis=1)
COST = prices_df[prices_df['symbol']=='COST'].drop('symbol', axis=1)
TMO = prices_df[prices_df['symbol']=='TMO'].drop('symbol', axis=1)
CSCO = prices_df[prices_df['symbol']=='CSCO'].drop('symbol', axis=1)
WMT = prices_df[prices_df['symbol']=='WMT'].drop('symbol', axis=1)
MCD = prices_df[prices_df['symbol']=='MCD'].drop('symbol', axis=1)
DIS = prices_df[prices_df['symbol']=='DIS'].drop('symbol', axis=1)
ABT = prices_df[prices_df['symbol']=='ABT'].drop('symbol', axis=1)
WFC = prices_df[prices_df['symbol']=='WFC'].drop('symbol', axis=1)
ACN = prices_df[prices_df['symbol']=='ACN'].drop('symbol', axis=1)
CMCSA = prices_df[prices_df['symbol']=='CMCSA'].drop('symbol', axis=1)
ADBE = prices_df[prices_df['symbol']=='ADBE'].drop('symbol', axis=1)
CRM = prices_df[prices_df['symbol']=='CRM'].drop('symbol', axis=1)
DHR = prices_df[prices_df['symbol']=='DHR'].drop('symbol', axis=1)
VZ = prices_df[prices_df['symbol']=='VZ'].drop('symbol', axis=1)
TXN = prices_df[prices_df['symbol']=='TXN'].drop('symbol', axis=1)
LIN = prices_df[prices_df['symbol']=='LIN'].drop('symbol', axis=1)
NKE = prices_df[prices_df['symbol']=='NKE'].drop('symbol', axis=1)
NFLX = prices_df[prices_df['symbol']=='NFLX'].drop('symbol', axis=1)
PM = prices_df[prices_df['symbol']=='PM'].drop('symbol', axis=1)
BMY = prices_df[prices_df['symbol']=='BMY'].drop('symbol', axis=1)
NEE = prices_df[prices_df['symbol']=='NEE'].drop('symbol', axis=1)
RTX = prices_df[prices_df['symbol']=='RTX'].drop('symbol', axis=1)
QCOM = prices_df[prices_df['symbol']=='QCOM'].drop('symbol', axis=1)
T = prices_df[prices_df['symbol']=='T'].drop('symbol', axis=1)


# Concatenate the ticker DataFrames
prices_df = pd.concat([
    BRK,
    MSFT,
    AMZN,
    GOOGL,
    MS,
    NVDA,
    TSLA,
    GOOG,
    XOM,
    UNH,
    JPM,
    JNJ,
    META,
    V,
    HD,
    PG,
    MA,
    CVX,
    MRK,
    ABBV,
    LLY,
    BAC,
    AVGO,
    PEP,
    PFE,
    KO,
    COST,
    TMO,
    CSCO,
    WMT,
    MCD,
    DIS,
    ABT,
    WFC,
    ACN,
    CMCSA,
    ADBE,
    CRM,
    DHR,
    VZ,
    TXN,
    LIN,
    NKE,
    NFLX,
    PM,
    BMY,
    NEE,
    RTX,
    QCOM,
    T], axis=1, keys=[
    "BRK.B",
    "MSFT",
    "AMZN",
    "GOOGL",
    "MS",
    "NVDA",
    "TSLA",
    "GOOG",
    "XOM",
    "UNH",
    "JPM",
    "JNJ",
    "META",
    "V",
    "HD",
    "PG",
    "MA",
    "CVX",
    "MRK",
    "ABBV",
    "LLY",
    "BAC",
    "AVGO",
    "PEP",
    "PFE",
    "KO",
    "COST",
    "TMO",
    "CSCO",
    "WMT",
    "MCD",
    "DIS",
    "ABT",
    "WFC",
    "ACN",
    "CMCSA",
    "ADBE",
    "CRM",
    "DHR",
    "VZ",
    "TXN",
    "LIN",
    "NKE",
    "NFLX",
    "PM",
    "BMY",
    "NEE",
    "RTX",
    "QCOM",
    "T"])


In [76]:
# Review the first 5 rows of the DataFrame
prices_df

BRK.B                                      \
                             open      high      low   close   volume   
timestamp                                                               
2017-05-01 04:00:00+00:00  165.80  166.3200  165.350  165.44  3135353   
2017-05-02 04:00:00+00:00  165.90  166.3299  165.160  166.05  2781230   
2017-05-03 04:00:00+00:00  165.70  167.0800  165.500  166.73  2982169   
2017-05-04 04:00:00+00:00  167.64  168.0400  166.220  166.34  3643844   
2017-05-05 04:00:00+00:00  167.12  167.1400  165.879  166.55  3578534   
...                           ...       ...      ...     ...      ...   
2020-04-27 04:00:00+00:00  187.78  189.3700  186.350  187.46  6625074   
2020-04-28 04:00:00+00:00  190.54  191.0000  187.000  187.12  6679292   
2020-04-29 04:00:00+00:00  190.64  192.6800  188.930  189.61  7654591   
2020-04-30 04:00:00+00:00  188.30  189.8700  186.710  187.36  8224436   
2020-05-01 04:00:00+00:00  185.21  185.8900  181.640  182.67  7516589   

                                                     MSFT                   \
                          trade_count        vwap    open    high      low   
timestamp                                                                    
2017-05-01 04:00:00+00:00       23981  165.746253   68.68   69.55   68.500   
2017-05-02 04:00:00+00:00       25620  165.858722   69.71   69.71   69.130   
2017-05-03 04:00:00+00:00       25626  166.595268   69.38   69.38   68.710   
2017-05-04 04:00:00+00:00       29600  166.869067   69.03   69.08   68.640   
2017-05-05 04:00:00+00:00       27209  166.331316   68.90   69.03   68.485   
...                               ...         ...     ...     ...      ...   
2020-04-27 04:00:00+00:00       62619  187.825859  176.59  176.90  173.300   
2020-04-28 04:00:00+00:00       67082  188.543306  175.59  175.67  169.390   
2020-04-29 04:00:00+00:00       74721  190.772239  173.22  177.68  171.880   
2020-04-30 04:00:00+00:00       77702  187.971737  180.00  180.40  176.230   
2020-05-01 04:00:00+00:00       90688  183.333253  175.80  178.64  174.010   

                           ...      QCOM                             T         \
                           ...    volume trade_count       vwap   open   high   
timestamp                  ...                                                  
2017-05-01 04:00:00+00:00  ...  12987659       74022  53.388232  39.68  39.70   
2017-05-02 04:00:00+00:00  ...  10445444       62192  53.294291  39.14  39.22   
2017-05-03 04:00:00+00:00  ...  18428284       85676  54.238582  38.94  38.98   
2017-05-04 04:00:00+00:00  ...  11998428       67111  54.799165  38.41  38.41   
2017-05-05 04:00:00+00:00  ...   9108818       49168  54.865637  38.08  38.71   
...                        ...       ...         ...        ...    ...    ...   
2020-04-27 04:00:00+00:00  ...   7219462       68928  76.366304  29.99  30.68   
2020-04-28 04:00:00+00:00  ...   6250473       54412  76.413261  31.00  31.12   
2020-04-29 04:00:00+00:00  ...  11951968       86985  78.574561  31.13  31.87   
2020-04-30 04:00:00+00:00  ...  17113450      121367  78.945041  31.00  31.05   
2020-05-01 04:00:00+00:00  ...   9742126       89780  75.689929  30.77  30.79   

                                                                           
                              low  close    volume trade_count       vwap  
timestamp                                                                  
2017-05-01 04:00:00+00:00  39.060  39.10  24750447       95238  39.278779  
2017-05-02 04:00:00+00:00  38.880  38.95  24852467       86237  38.996448  
2017-05-03 04:00:00+00:00  38.150  38.40  33743554      113920  38.423685  
2017-05-04 04:00:00+00:00  37.630  38.01  31450002      118191  37.965109  
2017-05-05 04:00:00+00:00  38.030  38.56  21829853       82070  38.424403  
...                           ...    ...       ...         ...        ...  
2020-04-27 04:00:00+00:00  29.740  30.54  33320892      146405  30.258582  
2020-04-28 04:00:

---

#### Step 4. Clean and Analyze API Data :

In [86]:
# Create and fill an empty closing_prices_df for the stock data. 
closing_prices_df = pd.DataFrame()

# Using a for loop, for every ticker in the tickers list, 
# Select the close price for each ticker in the prices_df Dataframe
# That will be set equal to closing_prices_df for the same ticker value
for ticker in tickers:
    closing_prices_df[ticker] = prices_df[ticker]["close"]

# For the new closing_prices_df DataFrame, keep only the date component
closing_prices_df.index = closing_prices_df.index.date

# Review the closing_prices_df DataFrame
display(closing_prices_df.head())



,BRK.B,MSFT,AMZN,GOOGL,MS,NVDA,TSLA,GOOG,XOM,UNH,...,TXN,LIN,NKE,NFLX,PM,BMY,NEE,RTX,QCOM,T
2017-05-01,165.44,69.41,948.23,932.82,43.74,106.64,322.83,912.57,82.06,174.59,...,79.29,NaN,54.99,155.35,110.55,56.31,133.04,118.92,53.60,39.10
2017-05-02,166.05,69.30,946.94,937.09,43.35,103.48,318.89,916.44,82.05,174.14,...,79.25,NaN,55.07,156.45,110.47,55.95,134.02,119.32,53.29,38.95
2017-05-03,166.73,69.08,941.03,948.45,43.48,104.25,311.02,927.04,82.70,172.54,...,79.77,NaN,54.53,155.59,110.61,55.06,133.36,120.30,54.49,38.40
2017-05-04,166.34,68.81,937.53,954.72,43.28,103.85,295.46,931.66,81.64,174.02,...,79.30,NaN,54.47,157.25,111.20,55.19,134.34,121.00,54.91,38.01
2017-05-05,166.55,69.00,934.15,950.28,43.08,103.86,308.35,927.13,82.02,174.12,...,79.63,NaN,53.95,156.60,112.25,55.15,134.33,121.35,54.93,38.56


In [87]:
# Compute daily returns of the closing_prices_df DataFrame using the pct_change function 
# Be sure to drop the first row of NaN values
daily_returns_df = closing_prices_df.pct_change().dropna()

# Review the resulting daily_returns_df DataFrame
display(daily_returns_df.head())



,BRK.B,MSFT,AMZN,GOOGL,MS,NVDA,TSLA,GOOG,XOM,UNH,...,TXN,LIN,NKE,NFLX,PM,BMY,NEE,RTX,QCOM,T
2018-11-01,0.002825,-0.008333,0.042253,-0.004218,0.005037,0.034530,0.020633,-0.006287,0.012425,-0.001607,...,0.051384,-0.020971,0.023321,0.051693,0.003179,0.031460,-0.011768,0.027131,0.013039,-0.006193
2018-11-02,0.003449,0.002266,0.000000,-0.013343,0.001307,-0.014626,0.006187,-0.011411,0.015867,0.000307,...,-0.026025,-0.013333,-0.002735,-0.026089,0.004527,-0.027623,-0.013609,-0.001176,-0.005965,0.000984
2018-11-05,0.046812,0.012717,-0.022653,-0.014708,-0.002829,-0.014657,-0.014463,-0.016733,-0.003783,0.000000,...,0.006312,0.013451,0.012405,0.020511,0.009014,0.026435,0.015225,-0.001413,0.002211,0.003604
2018-11-06,0.010498,0.001953,0.009221,0.013109,-0.002182,-0.003353,-0.000996,0.015114,0.004287,0.009693,...,0.015471,0.021853,-0.012382,-0.014583,-0.008710,-0.013069,0.004393,0.005422,0.002521,0.010774
2018-11-07,0.012494,0.039361,0.068590,0.036155,0.019466,0.012935,0.020817,0.035594,0.012684,0.042081,...,0.000618,0.007974,0.018284,0.053597,0.003380,0.031938,0.012131,0.020400,-0.006601,0.004522


In [85]:
# Using a for loop, for every ticker in the tickers list, 
# Select the close price for each ticker in the prices_df Dataframe
# That will be set equal to closing_prices_df for the same ticker value
closing_prices_df = pd.DataFrame()
for ticker in tickers:
    closing_prices_df[ticker] = prices_df[ticker]['close']
     
# For the new closing_prices_df DataFrame, keep only the date component
closing_prices_df.index = prices_df.index.date

daily_returns_df = closing_prices_df.pct_change()

daily_returns_df.describe()

,BRK.B,MSFT,AMZN,GOOGL,MS,NVDA,TSLA,GOOG,XOM,UNH,...,TXN,LIN,NKE,NFLX,PM,BMY,NEE,RTX,QCOM,T
count,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,...,756.000000,376.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000,756.000000
mean,0.000240,0.001860,0.009283,0.000622,0.000079,0.001788,0.001753,0.007953,-0.000697,0.000852,...,0.000643,0.000429,0.000766,0.001628,-0.000370,0.000229,0.000835,-0.000512,0.000747,-0.000222
std,0.014785,0.037970,0.252582,0.018161,0.022492,0.031399,0.038209,0.235074,0.017486,0.020243,...,0.020681,0.020055,0.019162,0.025591,0.018453,0.016571,0.015959,0.025293,0.024323,0.016275
min,-0.095921,-0.440421,-0.872469,-0.116342,-0.156000,-0.187559,-0.185778,-0.864151,-0.122248,-0.172769,...,-0.118257,-0.102833,-0.116301,-0.111375,-0.155757,-0.139424,-0.134171,-0.419486,-0.149453,-0.086677
25%,-0.005076,-0.005952,-0.007053,-0.006557,-0.008541,-0.012122,-0.016280,-0.006321,-0.007733,-0.006882,...,-0.008509,-0.008610,-0.007116,-0.011478,-0.007578,-0.007200,-0.005058,-0.005344,-0.009350,-0.006299
50%,0.000609,0.001379,0.001417,0.000961,0.000103,0.002036,0.000751,0.000858,-0.000242,0.000959,...,0.001641,0.001287,0.000801,0.000467,0.000182,0.000812,0.001425,0.001106,0.000613,0.000725
75%,0.006413,0.009579,0.010567,0.009213,0.009053,0.016986,0.018983,0.009276,0.006939,0.008399,...,0.010684,0.009404,0.009411,0.015752,0.008086,0.009100,0.006795,0.006893,0.009709,0.007183
max,0.116099,0.790947,6.868773,0.096202,0.197700,0.178259,0.198949,6.385257,0.126868,0.127989,...,0.135294,0.116947,0.151752,0.135436,0.082171,0.061207,0.136904,0.157572,0.232074,0.079211


## Part 2. Visualizing the Data :
Visualization and descriptions of the Data help us understand the data we are working with, and review the data for any faults/drawbacks. Generally, the larger the data-set, and the more data that is used, the more accurate the data will be! By using python tools, and visualization plots, we can infer whether data needs to be cleaned, may have innacuracies, or won't aid us in achieving our desired outcome. 



In [ ]:
# Visualizing the Dataframe, with different plots. Using .pct_change() to get the percentage
# Change in data values.
print("2. Visualizations of MD Data :")
print(" NOTE : 'pct_change' refers to the ROC or velocity of the charges data.")
print("")
print("")
# Creating a line, and bar plot visualization of MD charges data
display(md_cost_age_df.plot(
    kind='line', 
    y= "charges",
    figsize=(10,7),
    title = "Line Plot Visualization of Medical Insurance Charges Data"
))
display(md_cost_age_df.plot(
    kind='bar', 
    y= "charges",
    figsize=(10,7),
    title = "Bar Plot Visualization of Medical Insurance Charges Data"
    
))

--- 

In [ ]:
# Creating a bar plot visualizing the rate of change of medical insurance charges 
display(md_cost_age_df.pct_change().plot(
    kind='bar', 
    y= "charges",
    figsize=(10,7),
    title = "Bar Plot Visualization of 'pct_change' in Medical Insurance Charges - Based on User Age"
    
))

# Creating a histogram plot of the ROC data, to visualize the standard deviation of the data (a tool to help measure how accurate/correlated the data is) 
display(md_cost_age_df.pct_change().plot(
    kind='hist', 
    y= "charges",
    figsize=(10,7),
    title = "Histogram Plot Visualizing the Standard Deviation in Medical Insurance Charges 'pct_change' data, based on age"
    
))

## Part 2.5 Data Cleaning/Analysis : Cleaning and Reviewing the Data.
The acquired data needs some working with, as there is a high standard deviation in the pct_change() data of the dataset, and the correlation is not strong enough between increases in MD insurance costs. To solve this issue, I will find the average year-to-year increase from the entire dataset, instead of first focusing on age, and see if this changes the data outcome! 

In [ ]:
md_cost_age_df_1 = (
 medical_costs_df[["charges", "age"]]
 .groupby("age")
 .mean()
 .pct_change()
 .sort_values("age")
)

md_cost_age_df_median = md_cost_age_df_1.median()
md_cost_age_df_mean = md_cost_age_df_1.mean()

display(md_cost_age_df_1.describe())
display(print(f" the expected median pct_change in MD insurance charges will likely be close to: +{float(md_cost_age_df_median.tail(1))* 100}%"))
display(print(f" the expected mean (average) pct_change in MD insurance charges will likely be close to: +{float(md_cost_age_df_mean.tail(1)* 100)}%"))
print(f" Based on the data, an average annual pct_change() in total medical insurance costs, between the ages of 18 and 64 should end up at a roughly +{float(md_cost_age_df_mean.tail(1)* 100)}% annually")

print("Based on these results, the user should definetely look to invest sooner, so as not to miss out on the time-value of money. But they are also shown that even with expected charges, there remains uncertainty-- the ROC for MD insurance costs has a median value far less than the average, meaning there is a discrepency between the changes as we age, and between different individuals.")


---

Wrap up :

Though the data isn't 'perfect', the application with all its simulations can only help the user 'predict; how thir financials will look like, based on current Data. An iomprovement I would definetely consider making is pulling in a larger dataset for MD insurance costs. 

Why we need the data:

The data will allow us to predict the users future MD insurance charges, based on the expected pct_change in their costs as they get older.



## Working with Data - Finding the Optimal Portfolio Weight.

In [36]:
# Using thje MCForecastTools library, I can create monte carlo simulations, to try and predict how
# The portfolio will likely look like in 30 years time. This will be 
# Matched up with the user data, so that we can help meet his/her financial needs, specific
# To the age and current charges.
MC_thirty_year = MCSimulation(
    portfolio_data = prices_df,
    num_simulation = 2000,
    num_trading_days = 252*30
)


display(MC_thirty_year.calc_cumulative_return())

# Review the simulation input data
display(MC_thirty_year.portfolio_data.head())

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.


/Users/najibabounasr/Desktop/Berkley_Bootcamp/Project 1/Project_1_Portfolio_Curator-/MCForecastTools.py:117: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  portfolio_cumulative_returns[n] = (1 + sim_df.fillna(0)).cumprod()


Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo simulation number 230.
Running Monte Carlo simulation number 240.
Running Monte Carlo simulation number 250.
Running Monte Carlo simulation number 260.
Running Monte Carlo simulation number 270.
Running Monte Carlo simulation number 280.
Running Monte Carlo simulation number 290.
Running Monte Carlo simulation number 300.
Running Monte Carlo simulation number 310.
Running Monte Carlo simulation number 320.
Running Monte Carlo simulation number 330.
Running Mon

,0,1,2,3,4,5,6,7,8,9,...,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7556,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7557,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7558,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
7559,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


AAPL                                         \
                          open high low close volume trade_count vwap   
timestamp                                                               
2017-05-01 04:00:00+00:00  NaN  NaN NaN   NaN    NaN         NaN  NaN   
2017-05-02 04:00:00+00:00  NaN  NaN NaN   NaN    NaN         NaN  NaN   
2017-05-03 04:00:00+00:00  NaN  NaN NaN   NaN    NaN         NaN  NaN   
2017-05-04 04:00:00+00:00  NaN  NaN NaN   NaN    NaN         NaN  NaN   
2017-05-05 04:00:00+00:00  NaN  NaN NaN   NaN    NaN         NaN  NaN   

                                         MSFT         ...       QCOM  \
                          daily_return   open   high  ...       vwap   
timestamp                                             ...              
2017-05-01 04:00:00+00:00          NaN  68.68  69.55  ...  53.388232   
2017-05-02 04:00:00+00:00          NaN  69.71  69.71  ...  53.294291   
2017-05-03 04:00:00+00:00          NaN  69.38  69.38  ...  54.238582   
2017-05-04 04:00:00+00:00          NaN  69.03  69.08  ...  54.799165   
2017-05-05 04:00:00+00:00          NaN  68.90  69.03  ...  54.865637   

                                            T                                 \
                          daily_return   open   high    low  close    volume   
timestamp                                                                      
2017-05-01 04:00:00+00:00          NaN  39.68  39.70  39.06  39.10  24750447   
2017-05-02 04:00:00+00:00    -0.005784  39.14  39.22  38.88  38.95  24852467   
2017-05-03 04:00:00+00:00     0.022518  38.94  38.98  38.15  38.40  33743554   
2017-05-04 04:00:00+00:00     0.007708  38.41  38.41  37.63  38.01  31450002   
2017-05-05 04:00:00+00:00     0.000364  38.08  38.71  38.03  38.56  21829853   

                                                               
                          trade_count       vwap daily_return  
timestamp                                                      
2017-05-01 04:00:00+00:00       95238  39.278779          NaN  
2017-05-02 04:00:00+00:00       86237  38.996448    -0.003836  
2017-05-03 04:00:00+00:00      113920  38.423685    -0.014121  
2017-05-04 04:00:00+00:00      118191  37.965109    -0.010156  
2017-05-05 04:00:00+00:00       82070  38.424403     0.014470  

[5 rows x 400 columns]

In [ ]:
MC_thirty_summary_statistics = MC_thirty_year.summarize_cumulative_return()
# Read and display summary statistics
MC_thirty_summary_statistics


In [ ]:
MC_sim_line_plot = MC_thirty_year.plot_simulation()

Visualizations lets us understand the range of outcome the user
will likely face when choosing to purchase our financial product. The goal will be to maximize long-term stability and growth, with low volatility. This has ben made possible by screening our stocks

In [ ]:
# Visualize the probability distribution of the 30-year Monte Carlo simulation 
# by plotting a histogram - 
MC_sim_dist_plot = MC_thirty_year.plot_distribution()

print("The histogram predictive plot has shown that 

## Analyzing the Portfolio Stocks, Individually.

In [ ]:
print(" CALCULATION OF RISK UNDER ONE (1) STANDARD DEVIATION : ")
print("  ")


n = 1 
for ticker in tickers :
    ticker = str(input(
    mean = daily_returns_df[ticker].mean()
    std = daily_returns_df[ticker].std() 
    daily_return_probability_plus = (mean + std) * 100
    daily_return_probability_minus = (mean - std) * 100
    print(f"{n}.({ticker}) :  A 95% probability exists that the daily value of {ticker} will likely be the mean ({mean}) plus or minus two standard deviations ({std})")
    print("-------")
    print(f"   that gives us a range of : {daily_return_probability_minus}%")
    print(f"   to : +{daily_return_probability_plus}%")
    nf_lower_bound_return = mean - (1.96 * std)
    nf_upper_bound_return = mean + (1.96 * std)
    nn_lower_bound_return = mean - (2.576 * std)
    nn_upper_bound_return = mean + (2.576 * std)
    print("-------")
    print(f" From the analysis of the {ticker} historical return data, with a 95% confidence interval, we can expect our daily return value to fall between {nf_lower_bound_return*100}% and +{nf_upper_bound_return*100}% ")    
    print("-------")
    rec_value = closing_prices_df[ticker].tail(1)
    lower_bound_price = rec_value.values * (1 - nf_lower_bound_return)
    print(f"            The lower bound of the expected price for the next day is ${lower_bound_price} We get that from the following equation: ")
    print (f"           {closing_prices_df[ticker].tail(1).values} * 1 - {nf_lower_bound_return} ") 
    upper_bound_price = rec_value.values * (1 + nf_upper_bound_return)
    print(f"            The upper bound of the expected price for the next day is ${upper_bound_price} We get that from the following equation: ")
    print(f"            {closing_prices_df[ticker].tail(1).values} * 1 + {nf_lower_bound_return} ") 
    print(f"            {ticker}")
    print("")
    print("")
    n = n + 1

In [ ]:
print(" CALCULATION OF RISK UNDER THREE (3) STANDARD DEVIATIONS : ")
print("  ")

n = 1 
for ticker in tickers :  
    mean = daily_returns_df[ticker].mean()
    std = daily_returns_df[ticker].std() * 3
    daily_return_probability_plus = (mean + std) * 100
    daily_return_probability_minus = (mean - std) * 100
    print(f"{n}.({ticker}) :  For a 99.7% probability exists that the daily value of {ticker} will likely be the mean ({mean}) plus or minus two standard deviations ({std})")
    print(f"   that gives us a range of : {daily_return_probability_minus}%")
    print(f"   to : +{daily_return_probability_plus}%")
    print("---------------------")
    down_tail_percentage = (mean - ( std)) * 100
    print(f"The downside tail-percentage of {ticker} is : {down_tail_percentage}%")
    up_tail_percentage = (mean + ( std)) * 100
    print(f"The upside tail-percentage of {ticker} is : +{up_tail_percentage}%")
    print(f"   {ticker}")
    print("")
    print("")
    n = n + 1

In [ ]:
daily_returns_df.plot(kind='hist'
                     )

The (above) visualization will support our finding that the standard deviation of our portfolio is within an adequate range of values-- showing relatively low volatility. 

In [ ]:
initial_investment = 10000
ci_lower_ten_cumulative_return = MC_thirty_summary_statistics[8] * initial_investment
ci_upper_ten_cumulative_return = MC_thirty_summary_statistics[9] * initial_investment

# Print the result of your calculations
print(f"There is a 95% chance that an initial investment of $ {initial_investment} in the portfolio"
    f" over the next 30 years will end within in the range of"
    f" ${ci_lower_ten_cumulative_return :2f} and ${ci_upper_ten_cumulative_return :2f}.")



## Creating a Monte Carlo Simulation to Calculate Optimal Weight Distribution

In [2]:
from io import BytesIO

import numpy as np
import numba as nb
import pandas as pd
import holoviews as hv
import panel as pn
from pathlib import Path

from scipy.optimize import minimize


In [6]:
import module1 as md
md.random_allocation(prices_df)

NameError: name 'null' is not defined